<a href="https://colab.research.google.com/github/shairal1/Knowledge_graph_rag/blob/main/data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# CSX / extended ASCII mappings
csx_to_iast = {
    "à": "ā",  "â": "Ā",
    "ã": "ī",  "ä": "Ī",
    "å": "ū",  "æ": "Ū",
    "ç": "ṛ",  "è": "Ṛ",  "é": "ṝ",
    "ë": "ḷ",  "í": "ḹ",
    "ï": "ṅ",  "ð": "Ṅ",
    "¤": "ñ",  "¥": "Ñ",
    "ñ": "ṭ",  "ò": "Ṭ",
    "ó": "ḍ",  "ô": "Ḍ",
    "õ": "ṇ",  "ö": "Ṇ",
    "÷": "ś",  "ø": "Ś",
    "ù": "ṣ",  "ú": "Ṣ",
    "ü": "ṃ",  "ý": "Ṃ",
    "þ": "ḥ",
    "¹": "ē",  "º": "ō",
    "×": "ḷ",  "Ÿ": "ṛ",
    "­": "ṇ",  "É": "ḵ",  "Â": "ṭ̱"
}

# Under-dot ASCII mappings
underdot_to_iast = {
    "r.": "ṛ",
    "r..": "ṝ",
    "l.": "ḷ",
    "l..": "ḹ",
    "n.": "ṇ",
    "t.": "ṭ",
    "d.": "ḍ",
    "s.": "ṣ",
    "m.": "ṃ",
    "h.": "ḥ"
}

# Merge mappings for general use
def build_full_mapping():
    full_map = csx_to_iast.copy()
    # add under-dot ASCII mapping (single character replacement only)
    for k, v in underdot_to_iast.items():
        # treat 'r.' etc as two-char sequences
        full_map[k] = v
    return full_map

full_mapping = build_full_mapping()
import unicodedata
import re

def normalize_sanskrit(text):
    # Step 1: replace CSX / extended ASCII
    for k, v in csx_to_iast.items():
        text = text.replace(k, v)

    # Step 2: replace under-dot ASCII sequences (longest first)
    for k, v in sorted(underdot_to_iast.items(), key=lambda x: -len(x[0])):
        text = text.replace(k, v)

    # Step 3: normalize Unicode
    text = unicodedata.normalize('NFC', text)

    # Step 4: optional cleanup (extra spaces)
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [2]:
import requests

# URL of the text file
url = "https://gretil.sub.uni-goettingen.de/gretil/1_sanskr/2_epic/mbh/ext/bhg4c__c.txt"

# Path where you want to save the file
# Example Windows: "C:/Users/YourName/Documents/bhg4c__c.txt"
# Example Mac/Linux: "/Users/YourName/Documents/bhg4c__c.txt"
save_path = "/content/bhg4c__c.txt"

# Fetch the file
response = requests.get(url)

if response.status_code == 200:
    with open(save_path, "w", encoding="utf-8") as f:
        f.write(response.text)
    print(f"File saved successfully at {save_path}")
else:
    print(f"Failed to download. Status code: {response.status_code}")
with open('/content/bhg4c__c.txt', 'r', encoding='utf-8') as f:
    content = f.read()  # Reads the entire file into a string

# Print first 500 characters to check
print(content[:500])


Bhagavadgita
with the commentaries of Sridhara, (Madhusudana), Visvanatha and Baladeva

Input by ... (Gaudiya Grantha Mandira)


___________________________________________________________________

THIS TEXT FILE IS FOR REFERENCE PURPOSES ONLY!   
COPYRIGHT AND TERMS OF USAGE AS FOR SOURCE FILE. 
                                       
Text converted to Classical Sanskrit Extended (CSX) encoding:
                                       
description             character  =ASCII   
               


In [15]:
from os import path
import json
import os

def gretil(output_dir):
  output_file = os.path.join(output_dir, "gretil_extracted_BHG.json")
  clean_text = "\n".join(line for line in content.splitlines() if not line.startswith("[*ENDNOTE]"))
  clean_text = normalize_sanskrit(clean_text)

  chunk_split = re.split(r'(BhG \d+\.\d+)', clean_text)
  chunk_split.pop(0)
  verse_id=[]
  text=[]
  for i in range(0,len(chunk_split),2):
    verse_id.append((chunk_split[i]).strip())
    text.append(chunk_split[i+1].strip())
  nested_json = {}
  pattern = r'.*?\|\|\d+\|\|'
  for i in range(len(text)):
      chapter_number=verse_id[i]
      nested_json[chapter_number] = {"chapter": chapter_number, "verse": [],"commentaries":[]}
      split=re.findall(pattern, text[i], flags=re.DOTALL)
      nested_json[chapter_number]["verse"].append(split[0])
      nested_json[chapter_number]["commentaries"].append(split[1:])

  with open(f'{output_file}', "w", encoding="utf-8") as f:
    json.dump(nested_json, f, ensure_ascii=False, indent=4)

gretil("/content/drive/MyDrive/Project")

########### Book


In [16]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 60.0 MB/s eta 0:00:00


In [18]:
import pdfplumber

pdf_path = "/content/Bhagavad-gita-As-It-Is_repaired.pdf"
pdf_text = ""

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        pdf_text += page.extract_text() + "\n"

In [19]:
def clean_text(text: str) -> str:
    # Replace page breaks (\f), carriage returns (\r), multiple newlines, tabs
    text = text.replace('\f', ' ')       # page breaks
    text = text.replace('\r', ' ')       # carriage returns
    text = text.replace('\t', ' ')
    return text

entries = []
import re

def split_transliteration_and_meaning(block: str):
    lines = block.strip().split("\n")
    translit_lines = []
    meaning_lines = []

    for line in lines:
        # If line contains a word-gloss like "word—meaning"
        if re.search(r'\w+—', line):
            meaning_lines.append(line)
        else:
            translit_lines.append(line)

    transliteration = "\n".join(translit_lines).strip()
    meaning = "\n".join(meaning_lines).strip()

    return transliteration, meaning

# Split by text number (assuming they appear as "TEXT 22", "TEXT 23", etc.)
text_blocks = re.split(r'TEXT\s+(\d+)', pdf_text)

# text_blocks[0] is before the first TEXT, so skip it
for i in range(1, len(text_blocks), 2):
    text_number = int(text_blocks[i].strip())
    content = text_blocks[i+1].strip()

    # Extract sections
    # Assuming headers like "TRANSLATION" and "PURPORT" exist
    headerparts = content.split("TRANSLATION")[0].strip()
    translation = re.search(r'TRANSLATION\s*(.*?)(PURPORT|$)', content, re.DOTALL).group(1).strip()
    purport_match = re.search(r'PURPORT\s*(.*)', content, re.DOTALL)
    purport = purport_match.group(1).strip() if purport_match else ""

    headerparts = re.split(r'\)\)\s*\d+\s*\)\)', headerparts)[1]
    transliteration,word_meaning=split_transliteration_and_meaning(headerparts)




    # Optional: extract transliteration if exists (custom logic depending on PDF)

    entry = {
        "text_number": text_number,
        "verse": normalize_sanskrit(clean_text(transliteration)),
        "mean_word2word": normalize_sanskrit(clean_text(word_meaning)),  # fill if you can parse it
        "translation": normalize_sanskrit(clean_text(translation)),
        "purport": normalize_sanskrit(clean_text(purport))
    }

    entries.append(entry)

# Save to JSON

In [21]:
with open("/content/drive/MyDrive/A_C_Bhaktivedanta_BhG.json", "w", encoding="utf-8") as f:
    json.dump(entries, f, ensure_ascii=False, indent=4)


621 entries saved to structured_texts.json
